In [41]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import logging 
import yaml

In [42]:
with open("../cofig.yaml" , "r") as f:
    config = yaml.full_load(f)

In [45]:
# Configure logging
logging.basicConfig(
    filename=r"../Log/Data-Ingestion-To-Sql.log",
    level=logging.INFO,   
    format="%(asctime)s - %(levelname)s - %(message)s"
)

In [44]:
df = pd.read_csv(config['file_path']['raw_file'])

logging.info("Data loaded in the df")

In [33]:
df.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [34]:
String_hostname =  config['Database']['String_hostname']
String_database = config['Database']['String_database']
String_port = config['Database']['String_port']
String_username = config['Database']['String_username']
String_password = config['Database']['String_password']

logging.info("All the required credential is fetched from yaml file")

In [35]:
# pip install pymysql
logging.error("Oh no")

In [36]:
from logging import exception

try:
    engine = create_engine( f"mysql+pymysql://{String_username}:{String_password}@{String_hostname}:{String_port}/{String_database}")
    print("Successfully connected to DB")
    logging.info("Created Engine.")
    with engine.connect() as connection:
        print("Successfully connected to DB")
        logging.info("Created Engine and connected successfully.")
except exception as e:
    print(e)
    logging.error(f"{e}")
finally:
    engine.dispose()
    print("sql connection is closed.")

Successfully connected to DB
Successfully connected to DB
sql connection is closed.


In [37]:
df.to_sql(name= 'olist_order_payments' , if_exists= "replace" , con= engine )

103886

In [38]:
df.memory_usage(deep=True).sum() / 1024**2  # in MB

np.float64(17.814587593078613)

In [39]:
query = "select * from olist_order_payments;"

sql_df = pd.read_sql(query , engine)

In [40]:
sql_df

,index,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45
...,...,...,...,...,...,...
103881,103881,0406037ad97740d563a178ecc7a2075c,1,boleto,1,363.31
103882,103882,7b905861d7c825891d6347454ea7863f,1,credit_card,2,96.80
103883,103883,32609bbb3dd69b3c066a6860554a77bf,1,credit_card,1,47.77
103884,103884,b8b61059626efa996a60be9bb9320e10,1,credit_card,5,369.54


In [ ]:
sql_df.dtypes

index                     int64
order_id                 object
payment_sequential        int64
payment_type             object
payment_installments      int64
payment_value           float64
dtype: object

In [48]:
import os

for i in os.listdir("../Data/Raw/"):
    if i != "olist_order_payments_dataset.csv":
        print(i)

olist_customers_dataset.csv
olist_geolocation_dataset.csv
olist_orders_dataset.csv
olist_order_items_dataset.csv
olist_order_reviews_dataset.csv
olist_products_dataset.csv
olist_sellers_dataset.csv
product_category_name_translation.csv
